In [1]:
import sys
sys.path.append("../../") 
sys.path.append("../") 

In [2]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import max_error, mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from modelos import *

In [3]:
def calcular_metricas(y_true, y_predict):
    return {
        "MAX_ERROR": max_error(y_true, y_predict),
        "MEAN_SQ_ERROR": mean_squared_error(y_true, y_predict),
        "MEDIAN_ABS_ERROR": median_absolute_error(y_true, y_predict),
        "MEAN_ABS_ERROR": mean_absolute_error(y_true, y_predict),
    }

In [4]:
def heuristica(Vviento):
    """Definimos la heurística basada en la Escala de Beaufort"""
    if Vviento < 3:
        return 0.4
    elif Vviento < 7:
        return 0.6
    elif Vviento < 12:
        return 1
    elif Vviento < 20:
        return 1.5
    elif Vviento < 26:
        return 2
    elif Vviento < 36:
        return 2.5
    elif Vviento < 46:
        return 3.2
    elif Vviento < 56:
        return 4.2
    elif Vviento < 66:
        return 5.4
    elif Vviento < 76:
        return 6.5
    elif Vviento < 88:
        return 8
    elif Vviento < 101:
        return 10
    elif Vviento >= 101:
        return 12

## Datos nuevos

Cargamos los datos que hemos capturado hasta ahora

In [5]:
data = pd.read_csv("./clean/forecast_merged.csv")
data = data.dropna() #Quitamos todas aquellas filas que tengan algun na

In [6]:
X = data.drop(columns = "AlturaOlas")
X = X.drop(columns=["antelacion", 'anio', 'mes', 'dia', 'hora'])
y = data["AlturaOlas"]
X_model = X.copy()

Obtenemos la escala que hemos utilizado para entrenar el modelo

In [7]:
filename = '../modelos/modelo_definitivo/scaler.pkl'

with open(filename, 'rb') as file:
    scaler = pickle.load(file)


scaler

ColumnTransformer(remainder='passthrough',
                  transformers=[('num', StandardScaler(),
                                 ['moonphase', 'temp', 'dewPt', 'heat_index',
                                  'rh', 'pressure', 'vis', 'wc', 'feels_like',
                                  'uv_index', 'Vviento', 'PeriodoOlas',
                                  'Lluvia', 'Nubosidad'])])

In [8]:
X_model = scaler.transform(X_model)

Cargamos el modelo para poder comparar con la heuristica

In [9]:
with open("../modelos/modelo_definitivo/modelo_rf_def.pkl", 'rb') as file:
    model = pickle.load(file)
model

RandomForestRegressor(max_depth=28, min_samples_leaf=4, min_samples_split=4,
                      n_estimators=201, random_state=777)

Realizamos las predicciones

In [12]:
predicciones_modelo = model.predict(X_model)
predicciones_heuristica = X["Vviento"].apply(heuristica)

Analizamos los resultados

In [13]:
calcular_metricas(y, predicciones_modelo)

{'MAX_ERROR': 0.6023284875710251,
 'MEAN_SQ_ERROR': 0.03153249963646717,
 'MEDIAN_ABS_ERROR': 0.10605952411812813,
 'MEAN_ABS_ERROR': 0.13435569186852309}

In [14]:
calcular_metricas(y, predicciones_heuristica)

{'MAX_ERROR': 0.6,
 'MEAN_SQ_ERROR': 0.0699526066350711,
 'MEDIAN_ABS_ERROR': 0.09999999999999998,
 'MEAN_ABS_ERROR': 0.19763033175355446}

Como podemos observar el modelo nuestro se comporta algo mejor que la heurística, pero se comporta de manera parecida en general. Que nuestro modelo se comporte parecido a la heurística es algo que no teníamos previsto pero puede darse por el hecho de predecir la altura de las olas a partir de predicciones, o por el hecho de que hasta el momento tenemos muy pocos datos recolectados de predicciones.

In [16]:
y.max()

1.2

Como se puede observar, las olas de los días que hemos recolectado son muy bajas y ese puede ser el motivo por el cual la heurística tiene tan buen rendimiento.
Por eso vamos a medir el rendimiento de la heurística con los datos reales históricos. De esta manera la heurística va a tener que predecir datos de todos los tipos y no únicamente olas bajas.

## Datos históricos


##### Probamos con todos los datos para ver si los mejores resultados de la heurística se deben al pequeño número de datos del que disponemos

In [17]:
X, y = datos_full()
X = X.drop(columns=["Temperatura", 'wspd', 'anio', 'mes', 'dia', 'hora'])
predicciones_heuristica = X["Vviento"].apply(heuristica)

In [18]:
calcular_metricas(y, predicciones_heuristica)

{'MAX_ERROR': 4.4,
 'MEAN_SQ_ERROR': 0.48961046057767366,
 'MEDIAN_ABS_ERROR': 0.3999999999999999,
 'MEAN_ABS_ERROR': 0.5072209211553473}

Se puede observar que nuestra heurística funciona mucho peor en el momento en el que se encuentra con una variedad de datos muy grande.

Ahora bien, no sabemos cómo se comporta nuestro modelo para estos datos. Podemos suponer que bien, porque en su momento entrenamos el modelo con datos da train y en el test había un buen rendimiento, pero no podemos asegurarlo.

A partir de aquí tenemos dos opciones, medir el rendimiento con muchos datos de predicciones por los cuales tenemos que esperar a los correspondientes datos reales, y por otro lado, sacar datos reales de 2024 y medir el rendimiento del modelo global en esos datos.

